In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

#print('key:', OPENAI_API_KEY)

In [6]:
from openai import OpenAI

client =  OpenAI(api_key=OPENAI_API_KEY)

In [7]:
from openai import OpenAI

client =  OpenAI(api_key=OPENAI_API_KEY)

response = client.responses.create(
    model='gpt-4.1',
    input='스타워즈 시리즈의 다스베이더의 역할에 대해 한줄로 설벙해줘'
)

In [8]:
response.output_text

'다스 베이더는 스타워즈 시리즈에서 제국의 무자비한 사령관이자, 타락한 제다이 아나킨 스카이워커의 새로운 자아로, 은하계를 위협하는 상징적인 악역입니다.'

In [9]:
response = client.responses.create(
    model='gpt-4.1',
    instructions='당신은 영화평론가야!',
    input='스타워즈 시리즈의 다스베이더의 역할에 대해 한줄로 설명해줘'
)
print(response.output_text)

다스 베이더는 스타워즈 시리즈에서 과거의 영웅 아나킨 스카이워커가 타락해 은하 제국의 상징적 악당이자, 아버지로서 주인공의 운명을 뒤흔드는 비극적 존재이다.


In [10]:
response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role': 'developer',
            'content': '음식에 대한 이야기 하는것을 좋아해'
        },
        {
            'role': 'user',
            'content': '오늘은 무엇을 먹을까?'
        }
    ]
)
print(response.output_text)

좋은 질문이네요! 오늘은 어떤 음식을 먹고 싶은지 몇 가지 질문을 드려볼게요:

1. 한식, 양식, 중식, 일식 등 특정한 종류가 땡기나요?
2. 집에서 만들어 드실 건가요, 아니면 배달이나 외식 생각 중이신가요?
3. 가볍게 먹고 싶은지, 든든한 한 끼가 필요한지도 궁금해요.
4. 재료나 음식 알레르기, 피하고 싶은 음식이 있으신가요?

예를 들어, 오늘 기분이 맑고 상쾌하다면 샐러드나 샌드위치를 추천하고, 든든한 한식이 생각나면 김치찌개나 제육볶음, 혹은 찜닭도 좋아요. 입맛 따라 뭔가 특별히 해보고 싶은 메뉴가 있으신지도 말씀해주시면 맞춤 메뉴 추천드릴 수 있어요! 😊

혹시 생각나는 메뉴, 혹은 오늘의 컨디션 말씀해주실 수 있나요?


In [11]:
completion = client.chat.completions.create(
    model='gpt-4.1',
    messages= [
        {
            'role': 'user',
            'content': '스타워즈시리즈에 대해 한분장으로 설명해줘'
        }
    ]  
)


In [12]:
#dict(completion.choices[0])
completion.choices[0].message.content

'스타워즈 시리즈는 선과 악의 대립, 은하 제국과 반란군, 그리고 포스라는 신비한 힘을 중심으로 다양한 인물과 우주 세계관을 그린 SF 영화 시리즈입니다.'

In [13]:
user_input = input('안녕하세요. 무엇이 굼금하신가요? 질문을 남겨주세요!!!')
print('질문:', user_input)

response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role': 'developer',
            'content': '너는 유치원 선생님이야. 아이들을 대하는것처럼 이야기해줘'
        },
        {
            'role': 'user',
            'content': user_input
        }
    ]
)
print(response.output_text)

질문: 
알겠어요~! 우리 친구들 안녕하세요! 😊  
무엇이든 궁금한 게 있으면 선생님한테 물어봐도 좋아요.  
함께 재미있게 이야기해볼까요?  
무엇을 하고 싶나요~? 🐻🌈


In [14]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

ModuleNotFoundError: No module named 'requests'

In [ ]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
    "required": ["latitude", "longitude"],
    "additionalProperties": False
    },
    "strict": True
}]

In [ ]:
input_messages = [{ 'role': 'user', 'content':'오늘 파리 날씨 어때?'}]

# 함수 호출하는 기능
response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

In [ ]:
dict(response.output[0])

{'arguments': '{"latitude":48.8566,"longitude":2.3522}',
 'call_id': 'call_5akC5g1dyEDCKEpL40sMoiBY',
 'name': 'get_weather',
 'type': 'function_call',
 'id': 'fc_685bae49a6608198832c554beffde293067ceb33fda1e78d',
 'status': 'completed'}

In [ ]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args["latitude"], args["longitude"])
print(result)

23.3


In [ ]:
input_messages.append(tool_call)
input_messages.append({
    'type' : 'function_call_output',
    'call_id': tool_call.call_id,
    'output': str(result)
})

# 최종 응답
response2 = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

print(response2.output_text)

오늘 파리의 현재 기온은 약 23.3°C입니다. 날씨가 꽤 온화하고 쾌적한 편이네요! 더 자세한 정보(예: 강수나 바람 등)가 필요하시면 말씀해 주세요.


In [ ]:
from tools.basic_tools import calculate_age, convert_currency, calculate_bmi

In [ ]:
fn_tools = [
    {
        "type": "function",
        "name": "calculate_age",
        "description": "입력된 생년월일(YYYY-mm-dd)로 만 나이를 계산합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "birthdate": {"type": "string", "description": "생년월일, 형식:YYYY-MM-DD"}
            },
        "required": ["birthdate"],
        "additionalProperties": False
        },
        "strict": True
    },
    {
        "type": "function",
        "name": "convert_currency",
        "description": "입력된 달러(USD)를 원화(KRW)로 변환합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {"type": "number", "description": "달러(USD) 금액"}
            },
        "required": ["amount"],
        "additionalProperties": False
        },
        "strict": True
    },
    {
        "type": "function",
        "name": "calculate_bmi",
        "description": "키(cm), 몸무게(kg) 정보를 받아서 BMI를 계산합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "height": {"type": "number", "description": "키(cm)"},
                "weight": {"type": "number", "description": "몸무게(cm)"}
            },
        "required": ["height", "weight"],
        "additionalProperties": False
        },
        "strict": True
    }
]

In [ ]:
input_messages = [
    {
        "role" : "user",
        "content": "내 생일은 1990-01-01 이고, 100 달러를 가지고 있는데 원화로 변환해줘, 그리고 내 키는 190cm 이고 몸무게는 50kg 이야 BMI 점수를 알려줘"
    }
]

response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=fn_tools
)

print(response.output)
print(response.output_text)

[ResponseFunctionToolCall(arguments='{"birthdate":"1990-01-01"}', call_id='call_HpSpiX94iMziDUQRj7RuDGDo', name='calculate_age', type='function_call', id='fc_685caa7beca081998a22d32a26c686a003f75c4e5cee4aab', status='completed'), ResponseFunctionToolCall(arguments='{"amount":100}', call_id='call_SI9eZ0fjUQtas872kGzdvvRF', name='convert_currency', type='function_call', id='fc_685caa7c24288199918bc5c74178c9b103f75c4e5cee4aab', status='completed'), ResponseFunctionToolCall(arguments='{"height":190,"weight":50}', call_id='call_BJKCubcO3Fyfr4jpBlgRiOHa', name='calculate_bmi', type='function_call', id='fc_685caa7c31408199b4edaee8cc35b3bd03f75c4e5cee4aab', status='completed')]



In [ ]:
print(response.output[0])
print(response.output[1])
print(response.output[2])

ResponseFunctionToolCall(arguments='{"birthdate":"1990-01-01"}', call_id='call_HpSpiX94iMziDUQRj7RuDGDo', name='calculate_age', type='function_call', id='fc_685caa7beca081998a22d32a26c686a003f75c4e5cee4aab', status='completed')
ResponseFunctionToolCall(arguments='{"amount":100}', call_id='call_SI9eZ0fjUQtas872kGzdvvRF', name='convert_currency', type='function_call', id='fc_685caa7c24288199918bc5c74178c9b103f75c4e5cee4aab', status='completed')
ResponseFunctionToolCall(arguments='{"height":190,"weight":50}', call_id='call_BJKCubcO3Fyfr4jpBlgRiOHa', name='calculate_bmi', type='function_call', id='fc_685caa7c31408199b4edaee8cc35b3bd03f75c4e5cee4aab', status='completed')


In [ ]:
import json

if response.output :
    for tool_call in response.output:
        if tool_call.type == 'function_call':
            if tool_call.name == 'calculate_age':
                print(f'호출 도구:{tool_call.name}')
                print(f'매개변수: {tool_call.arguments}')
                # 매개변수 추출
                args = json.loads(tool_call.arguments)
                # 함수 실행 결과
                result = calculate_age(args['birthdate'])
                input_messages.append(tool_call)
                input_messages.append({
                    "type" : "function_call_output",
                    "call_id": tool_call.call_id,
                    "output": str(result)
                })
            
            
            if tool_call.name == 'convert_currency':
                print(f'호출 도구:{tool_call.name}')
                print(f'매개변수: {tool_call.arguments}')

                # 매개변수 추출
                args = json.loads(tool_call.arguments)

                # 함수 실행 결과
                result = convert_currency(args['amount'])

                input_messages.append(tool_call)
                input_messages.append({
                    "type" : "function_call_output",
                    "call_id": tool_call.call_id,
                    "output": str(result)
                })
            
            if tool_call.name == 'calculate_bmi':
                print(f'호출 도구:{tool_call.name}')
                print(f'매개변수: {tool_call.arguments}')

                # 매개변수 추출
                args = json.loads(tool_call.arguments)

                # 함수 실행 결과
                result = calculate_bmi(args['height'],args['weight'])

                input_messages.append(tool_call)
                input_messages.append({
                    "type" : "function_call_output",
                    "call_id": tool_call.call_id,
                    "output": str(result)
                })

print(input_messages)
            

호출 도구:calculate_age
매개변수: {"birthdate":"1990-01-01"}
호출 도구:convert_currency
매개변수: {"amount":100}
호출 도구:calculate_bmi
매개변수: {"height":190,"weight":50}
[{'role': 'user', 'content': '내 생일은 1990-01-01 이고, 100 달러를 가지고 있는데 원화로 변환해줘, 그리고 내 키는 190cm 이고 몸무게는 50kg 이야 BMI 점수를 알려줘'}, ResponseFunctionToolCall(arguments='{"birthdate":"1990-01-01"}', call_id='call_HpSpiX94iMziDUQRj7RuDGDo', name='calculate_age', type='function_call', id='fc_685caa7beca081998a22d32a26c686a003f75c4e5cee4aab', status='completed'), {'type': 'function_call_output', 'call_id': 'call_HpSpiX94iMziDUQRj7RuDGDo', 'output': '35'}, ResponseFunctionToolCall(arguments='{"amount":100}', call_id='call_SI9eZ0fjUQtas872kGzdvvRF', name='convert_currency', type='function_call', id='fc_685caa7c24288199918bc5c74178c9b103f75c4e5cee4aab', status='completed'), {'type': 'function_call_output', 'call_id': 'call_SI9eZ0fjUQtas872kGzdvvRF', 'output': '133000'}, ResponseFunctionToolCall(arguments='{"height":190,"weight":50}', call_id='call_BJKCubc

In [ ]:
response_msg = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=fn_tools
)

print(response_msg.output_text)

- 만 나이는 35세입니다.
- 100달러는 현재 환율로 약 133,000원입니다.
- 키 190cm, 몸무게 50kg의 BMI는 약 13.85로, 저체중에 해당합니다. 

추가 정보가 필요하거나 궁금한 점 있으면 말씀해 주세요!


### File Search

In [ ]:
import requests
from io import BytesIO
#from openai import OpenAI

#client = OpenAI()

def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    return result.id

In [ ]:
file_id = create_file(client, './howto-sockets.pdf')

file-JVeHeoikzPe3ZdafsP2DLH


In [ ]:
vector_store = client.vector_stores.create(
    name='knowledge_base'
)

print(vector_store.id)

vs_685cb68d844c81918e0fb0ff7a57e2ff


In [ ]:
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)

VectorStoreFile(id='file-JVeHeoikzPe3ZdafsP2DLH', created_at=1750906661, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_685cb68d844c81918e0fb0ff7a57e2ff', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [ ]:
result_list = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)

print(result_list)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-JVeHeoikzPe3ZdafsP2DLH', created_at=1750906661, last_error=None, object='vector_store.file', status='completed', usage_bytes=29112, vector_store_id='vs_685cb68d844c81918e0fb0ff7a57e2ff', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-JVeHeoikzPe3ZdafsP2DLH', last_id='file-JVeHeoikzPe3ZdafsP2DLH')


In [ ]:
response = client.responses.create(
    model='gpt-4.1',
    input='파이썬 코드로 소켓 만드는 방법을 간단하게 설명해줘',
    tools=[{
        "type" : "file_search",
        "vector_store_ids" : [vector_store.id]
    }],
    tool_choice='required'
)

print(response)

Response(id='resp_685cd0d3fcdc8199ac3bbff06b88803f05f92a27e588b3bc', created_at=1750913236.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_685cd0d465fc8199a5ff176733a0aade05f92a27e588b3bc', queries=['파이썬 코드로 소켓 만드는 방법을 간단하게 설명해줘', '파이썬 소켓 예시', '파이썬 소켓 생성', '파이썬 socket 사용법', '파이썬 서버 클라이언트 예제'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_685cd0d6e42c8199aa955f44b338768f05f92a27e588b3bc', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-JVeHeoikzPe3ZdafsP2DLH', filename='howto-sockets.pdf', index=1087, type='file_citation')], text='파이썬에서 소켓을 만드는 방법을 간단하게 정리하면 다음과 같습니다.\n\n### 1. 클라이언트 소켓 만들기\n\n```python\nimport socket\n\n# 1. 소켓 객체 생성 (AF_INET = IPv4, SOCK_STREAM = TCP)\ns = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\n# 2. 서버에 연결 (호스트, 포트)\ns.connect((\'www.example.com\', 80))\n# 3. 데이터 송신/수신 (e

In [ ]:
print(dict(response))

{'id': 'resp_685cd034f590819ab88a10e0baebd37300b65382008f721d', 'created_at': 1750913076.0, 'error': None, 'incomplete_details': None, 'instructions': None, 'metadata': {}, 'model': 'gpt-4.1-2025-04-14', 'object': 'response', 'output': [ResponseOutputMessage(id='msg_685cd0356944819a82e31b131382838900b65382008f721d', content=[ResponseOutputText(annotations=[], text="파이썬에서 소켓을 만드는 방법은 비교적 간단합니다. 기본적으로 socket 모듈을 사용합니다. 아래는 간단한 과정과 예시입니다.\n\n### 기본 순서\n\n1. socket 모듈 임포트\n2. 소켓 객체 생성 (`socket.socket()`)\n3. (서버의 경우) 소켓에 주소와 포트 할당 (`bind()`)\n4. (서버의 경우) 클라이언트의 연결 요청 대기 (`listen()`)\n5. (서버의 경우) 클라이언트의 연결 수락 (`accept()`)\n6. 데이터 송수신 (`send()`, `recv()`)\n7. 소켓 닫기 (`close()`)\n\n### TCP 서버 예제\n```python\nimport socket\n\n# 1. 소켓 객체 생성 (AF_INET: IPv4, SOCK_STREAM: TCP)\nserver_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)\n\n# 2. 주소와 포트 할당\nserver_socket.bind(('localhost', 12345))\n\n# 3. 연결 요청 대기\nserver_socket.listen(1)\nprint('서버 대기 중...')\n\n# 4. 연결 수락\nclient_socket, addr =

In [15]:
pip install streamlit

  Using cached streamlit-1.46.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.1.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pandas-2.3.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached pillow-11.2.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (8.9 kB)
  Using cached pyarrow-20.0.0-cp39-cp39-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.6-py3-none